In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [6]:
# prompt_template = "You are an experienced medical professional. Answer the following medial question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: {}"

# eval_dataset['prompt'] = eval_dataset.apply(lambda x: prompt_template.format(x['Open-ended Verifiable Question']), axis=1)
# eval_dataset.prompt[0]

In [7]:
# from datasets import Dataset
# ds = Dataset.from_pandas(eval_dataset[['prompt']])
# ds

## Set up model definition

In [8]:
# with open('/content/drive/My Drive/w266/project/266 Final Project/llama3pt2-1b-peft/adapter_config.json', 'r') as f:
#   l = f.readlines()

In [9]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-bootstrap"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     token=userdata.get('HF_TOKEN'))

In [10]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [11]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 32) model

In [13]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('notebook2'))


adapter_config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-output-reformatted-bootstrap-output.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [15]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman, who has been taking naproxen for osteoarthritis, is a gastric ulcer caused by her long-term use of the nonsteroidal anti-inflammatory drug (NSAID), naproxen. This NSAID can irritate the stomach lining, leading to the formation of ulcers and the subsequent vomiting of coffee-ground-like material.		Rating my answer as 1) Correct.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman with osteoarthritis, who is experiencing mild epigastric discomfort and vomiting coffee-ground-like material, is a peptic ulcer. Her use of naproxen can contribute to this condition by irritating the stomach and duodenal lining, potentially causing ulcers that may lead to bleeding. This is especially concerning due to her age, making peptic ulcer as the most probable cause. Therefore, the most likely cause of her gastrointestinal blood loss is a peptic ulcer. My confidence in this answer is: 1) Correct.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The most likely diagnosis for the 23-year-old male with recurrent scaly lesions on the glans penis, characterized by their consistent location and post-healing pigmentation, is a condyloma acuminatum, which is commonly referred to as a genital wart. These warts are caused by the human papillomavirus (HPV) and typically appear on the genitals. They can return in the same location, and their resolution may result in some discoloration, matching the symptoms described.		The correctness of my answer is rated as 1) Correct.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the description of the symptoms—recurrent scaly lesions on the glans penis that always appear at the same location, heal with slight hyperpigmentation, and are consistent in their location—the most likely diagnosis for this condition is fixed detedos of Bowen’s disease. This condition often involves recurrent lesions that are confined to the same spot, and it aligns well with the presentation of symptoms provided.		Rating: 1) Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition is fixed condyloma acuminatum. This is a benign, recurrent wart-like lesion on the penis that tends to appear at the same location each time it recurs. The healing often leaves slight hyperpigmentation.		Rate: 1) Correct
The most likely diagnosis for the condition described is fixed drug eruption (FDE). FDE is a dermatological condition that typically manifests with recurrent, well-defined lesions at the same location every time, and it can lead to post-inflammatory hyperpigmentation (PIH). This fits the scenario of the recurrent scaly lesions on the glans penis that resolve with slight hyperpigmentation. 		I rate the correctness of this answer as 1) Correct.
